In [1]:
import sys
sys.path.insert(0, '/***/***/pi/***/***/Graph_expressivity/')
import argparse
import pickle
import torch
from pathlib import Path
import numpy as np
import yaml
# from src.cross_validate import eval_cv, load_config_from_yaml
from experiments.classifier2 import PDEClassifier
from experiments.train import train, test
from torch_geometric.loader import DataLoader as DataListLoader
from torch.optim import Adam
import copy

In [2]:
root_path = '/***/***/pi/***/***/Graph_expressivity'
# args_w_bn = load_config_from_yaml('test_w_bn.yml')
# args_wo_bn = load_config_from_yaml('test_wo_bn.yml')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
data_path = f"{root_path}/data/ENZYMES"
result_path = f"{root_path}/results_test/ENZYMES"

with open(f"{data_path}/id_folds.pkl", "rb") as f:
    id_folds = pickle.load(f)
with open(f"{data_path}/dataset.pkl", "rb") as f:
    dataset = pickle.load(f)
n_input = dataset[0].x.size(1)
n_output = len(np.unique([data.y.item() for data in dataset]))

batch_size = 2048
train_idx, val_idx = id_folds[0]
train_set = [dataset[i] for i in train_idx]
val_set = [dataset[i] for i in val_idx]
train_loader = DataListLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataListLoader(val_set, batch_size=batch_size, shuffle=False)

In [4]:
pde_type = 'heat'
time_points = 10
time_range_start = 0
time_range = 4
hidden_units = 16
num_pde_layers = 2
num_lin_layers_between = 1
num_lin_layers_after = 2
p_dropout = 0.5
skip_conn = False
batch_norm = True
with open(f"{data_path}/id_folds.pkl", "rb") as f:
    id_folds = pickle.load(f)
with open(f"{data_path}/dataset.pkl", "rb") as f:
    dataset = pickle.load(f)
n_input = dataset[0].x.size(1)
n_output = len(np.unique([data.y.item() for data in dataset]))
ts = torch.linspace(time_range_start, time_range, time_points, dtype=torch.float, device=device)
model = PDEClassifier(
    pde=pde_type, 
    ts=ts, 
    n_input=n_input, 
    n_hidden=hidden_units, 
    n_output=n_output, 
    device=device,
    num_layers=num_pde_layers,
    num_lin_layers_between_pde=num_lin_layers_between,
    num_lin_layers_after_pde=num_lin_layers_after,
    p_dropout=p_dropout,
    skip_conn=skip_conn,
    batch_norm=batch_norm
).to(device)
initial_state_dict = copy.deepcopy(model.state_dict())

In [5]:
optimizer = Adam(model.parameters(), lr=1e-3)
# criterion = CrossEntropyLoss()
criterion = torch.nn.NLLLoss()
# Train the model
train(model, train_loader, optimizer, criterion, device, num_epochs=1000)
test(model, val_loader, device)

Epoch 1/1000, Loss: 1.9375, Accuracy: 0.1648
Epoch 101/1000, Loss: 1.6989, Accuracy: 0.2593
Epoch 201/1000, Loss: 1.5729, Accuracy: 0.3463
Epoch 301/1000, Loss: 1.4098, Accuracy: 0.4278
Epoch 401/1000, Loss: 1.3465, Accuracy: 0.4370
Epoch 501/1000, Loss: 1.2242, Accuracy: 0.5074
Epoch 601/1000, Loss: 1.1543, Accuracy: 0.5222
Epoch 701/1000, Loss: 1.0565, Accuracy: 0.5722
Epoch 801/1000, Loss: 1.0600, Accuracy: 0.5926
Epoch 901/1000, Loss: 1.0589, Accuracy: 0.5722
Test accuracy: 61.67%


Test reset parameters (should reset for a new cv fold)

In [6]:
middle_state_dict = copy.deepcopy(model.state_dict())
model.reset_parameters()
reset_state_dict = model.state_dict()
# check if the parameters have been reset
for layer in initial_state_dict:
    if not torch.all(torch.eq(initial_state_dict[layer], reset_state_dict[layer])):
        print(f"{layer} has not been reset")
    else:
        print(f"{layer} has been reset")

pde_layers.0.lin_layer.weight has not been reset
pde_layers.0.lin_layer.bias has not been reset
pde_layers.0.bn1.weight has been reset
pde_layers.0.bn1.bias has been reset
pde_layers.0.bn1.running_mean has been reset
pde_layers.0.bn1.running_var has been reset
pde_layers.0.bn1.num_batches_tracked has been reset
pde_layers.0.bn2.weight has been reset
pde_layers.0.bn2.bias has been reset
pde_layers.0.bn2.running_mean has been reset
pde_layers.0.bn2.running_var has been reset
pde_layers.0.bn2.num_batches_tracked has been reset
pde_layers.1.lin_layer.weight has not been reset
pde_layers.1.lin_layer.bias has not been reset
pde_layers.1.bn1.weight has been reset
pde_layers.1.bn1.bias has been reset
pde_layers.1.bn1.running_mean has been reset
pde_layers.1.bn1.running_var has been reset
pde_layers.1.bn1.num_batches_tracked has been reset
pde_layers.1.bn2.weight has been reset
pde_layers.1.bn2.bias has been reset
pde_layers.1.bn2.running_mean has been reset
pde_layers.1.bn2.running_var has bee

In [7]:
for layer in initial_state_dict:
    if not torch.all(torch.eq(middle_state_dict[layer], reset_state_dict[layer])):
        print(f"{layer} not eql")
    else:
        print(f"{layer} eql")

pde_layers.0.lin_layer.weight not eql
pde_layers.0.lin_layer.bias not eql
pde_layers.0.bn1.weight not eql
pde_layers.0.bn1.bias not eql
pde_layers.0.bn1.running_mean not eql
pde_layers.0.bn1.running_var not eql
pde_layers.0.bn1.num_batches_tracked not eql
pde_layers.0.bn2.weight not eql
pde_layers.0.bn2.bias not eql
pde_layers.0.bn2.running_mean not eql
pde_layers.0.bn2.running_var not eql
pde_layers.0.bn2.num_batches_tracked not eql
pde_layers.1.lin_layer.weight not eql
pde_layers.1.lin_layer.bias not eql
pde_layers.1.bn1.weight not eql
pde_layers.1.bn1.bias not eql
pde_layers.1.bn1.running_mean not eql
pde_layers.1.bn1.running_var not eql
pde_layers.1.bn1.num_batches_tracked not eql
pde_layers.1.bn2.weight not eql
pde_layers.1.bn2.bias not eql
pde_layers.1.bn2.running_mean not eql
pde_layers.1.bn2.running_var not eql
pde_layers.1.bn2.num_batches_tracked not eql
lin_layers_after_pde.0.lin_layer.weight not eql
lin_layers_after_pde.0.lin_layer.bias not eql
lin_layers_after_pde.0.bn.weig

In [5]:
model = PDEClassifier(
    pde='wave', 
    ts=ts, 
    n_input=n_input, 
    n_hidden=hidden_units, 
    n_output=n_output, 
    device=device,
    num_layers=num_pde_layers,
    num_lin_layers_between_pde=num_lin_layers_between,
    num_lin_layers_after_pde=num_lin_layers_after,
    p_dropout=p_dropout,
    skip_conn=skip_conn,
    batch_norm=False
).to(device)

In [6]:
optimizer = Adam(model.parameters(), lr=1e-3)
# criterion = CrossEntropyLoss()
criterion = torch.nn.NLLLoss()
# Train the model
train(model, train_loader, optimizer, criterion, device, num_epochs=1000)
test(model, val_loader, device)

Epoch 1/1000, Loss: 5.1978, Accuracy: 0.1444
Epoch 101/1000, Loss: 1.7534, Accuracy: 0.1907
Epoch 201/1000, Loss: 1.7516, Accuracy: 0.2000
Epoch 301/1000, Loss: 1.7527, Accuracy: 0.2259
Epoch 401/1000, Loss: 1.7083, Accuracy: 0.2685
Epoch 501/1000, Loss: 1.6975, Accuracy: 0.2352
Epoch 601/1000, Loss: 1.6881, Accuracy: 0.2574
Epoch 701/1000, Loss: 1.6512, Accuracy: 0.2889
Epoch 801/1000, Loss: 1.6412, Accuracy: 0.3056
Epoch 901/1000, Loss: 1.6392, Accuracy: 0.3204
Test accuracy: 25.00%
